# L'optimisation ☕️☕️

L'objectif de cette séquence va être de construire des méthodes et des algorithmes permettant de trouver des minimum locaux et/ou globaux de fonctions. Soit $f:\mathbb{R}^d\mapsto\mathbb{R}$, nous souhaitons résoudre (ou du moins sur une partie du domaine de définition de la fonction $f$) le problème suivant :

$$x^\star=\text{argmin}_{x\in\mathbb{R}^d}f(x).$$

## Imports et fonction de plot

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

In [ ]:
# Le code ci-dessous permettra d'afficher notre fonction à optimiser

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('ggplot')

def plot_loss_contour(obj_func, param_trace=None, figsize=None, three_dim=False, rotate=12, 
                      starting=None, title=None):
    
    x, y = np.mgrid[slice(-5, 5 + 0.1, 0.1),
                    slice(-5, 5 + 0.1, 0.1)]
    z = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i, j] = obj_func([x[i, j], y[i, j]])
    if figsize is not None:
        f = plt.figure(figsize=figsize)
    else:
        f = plt.figure(figsize=(12.0, 8.0))
    if three_dim:
        ax = f.gca(projection='3d')
    else:
        ax = f.gca()
    
    if three_dim:
        m = ax.plot_surface(x, y, z, cmap=cm.viridis)
    else:
        m = ax.contourf(x, y, z, levels = 15)
    #
    if param_trace is not None:
        if three_dim:
            eps = 0.5
            #ax.scatter(param_trace[:, 0], param_trace[:, 1], param_trace[:, 2] + eps, 
            #           color='blue', alpha=1)
            ax.plot(param_trace[:, 0], param_trace[:, 1], param_trace[:, 2] + eps, 
                    color='red')
            ax.view_init(65, rotate)
                
        else:
            if type(param_trace) is not tuple:
                param_trace = [param_trace]
            for p in param_trace:
                p = np.array(p) if type(p) is list else p
                plt.plot(p[:, 0], p[:, 1])
                plt.scatter(p[:, 0], p[:, 1])
            f.colorbar(m)
    if starting is not None:
        if three_dim:
            z = obj_func(starting)
            plt.plot([starting[0], starting[0]], [starting[1], starting[1]], [z, z+0.1], lw=4, 
                     color='red', label='The start of our optimization')
            plt.legend()
        else:
            plt.scatter(starting[0], starting[1], color='red', 
                        label='The start of our optimization')
            plt.legend()
    if title is not None:
        plt.title(title)
    #plt.show()

## I. La descente de gradient

Considérons la fonction suivante qui admet plusieurs minimums locaux.

In [ ]:
def f(x):
    x, y = x
    return np.sqrt((x**2 + y - 11)**2 + (x + y**2 - 7)**2)

In [ ]:
plot_loss_contour(f, three_dim=False, starting=[0, 2], 
                  title='2D heatmap of the function we want to optimize')

Il est également possible de la représenter en 3 dimensions.

In [ ]:
plot_loss_contour(f, three_dim=True, starting=[4, 4], 
                  title='3D heatmap of the function we want to optimize')

Dit autrement, nous considérons $f:\mathbb{R}^2\mapsto\mathbb{R}$ définie par $f(x, y)=\sqrt{(x^2+y-11)^2+(x+y^2-7)^2}$. $f$ est continue et infiniment dérivable.

Nous avons en particulier les dérivées partielles suivantes : 

\begin{equation}
\frac{\partial f}{\partial x}(x, y)=\frac{2x^3+x(2y-21)+y^2-7}{\sqrt{(x^2+y-11)^2+(x+y^2-7)^2}}
\end{equation}

et

\begin{equation}
\frac{\partial f}{\partial y}(x, y)=\frac{x^2+2xy+2y^3-13y-11}{\sqrt{(x^2+y-11)^2+(x+y^2-7)^2}}.
\end{equation}

Sans hypothèse sur la fonction $f$, celle-ci peut être très difficile à minimiser. Soit $x^{(0)}\in\mathbb{R}^2$, un algorithme permettant de chercher un minimum local en partant de $x^{(0)}$ est la descente de gradient. Ce dernier suppose que nous avons accès aux informations du premier ordre : le gradient $\nabla f(x, y)$. Rappelons que le gradient est le vecteur construit à partir des dérivées partielles $\nabla f (x, y) = [\partial f(x,y)/\partial x, \partial f(x, y)/\partial y]^T$. Ce dernier donne le sens de la plus forte croissance de la fonction $f$. Son opposé donne la plus forte pente. L'idée de l'algorithme de descente de gradient est de suivre la direction donnée par ce dernier par petits pas. On note $\boldsymbol{x}=(x,y)$. Nous avons ainsi :

$$\boldsymbol{x}^{(t+1)}=\boldsymbol{x}^{(t)}-\eta\nabla f(\boldsymbol{x}^{(t)})$$

où $\eta>0$ est justement un paramètre permettant de contrôler la taille du pas.

---

<span style="color:blue">**Exercice :**</span> **Donnez le code permettant de calculer le gradient de la fonction précédente (en format vecteur ligne).**



---

In [ ]:
def grad(x):
    ####### Complete this part ######## or die ####################
    v = f(x)
    x, y = x
    return np.array([
        (2*x**3+x*(2*y-21)+y**2-7)/v,
        (x**2+2*x*y+2*y**3-13*y-11)/v
    ])
    ###############################################################

---

<span style="color:blue">**Exercice :**</span> **Donnez le code permettant de calculer une itération de l'algorithme de descente de gradient. Attention, on appelle le pas d'optimisation $\eta$ le *learning rate*.**



---

In [ ]:
class GradientDescent(object):
    def optimize(self, learning_rate = 0.1, nb_iterations=15, beta=None):
        # beta est notre variable ! 
        # si elle n'est pas fixée on la tire au hasard
        if beta is None:
            beta = np.random.uniform(-2, 2, size=2)

        param_trace = [beta]
        loss_trace = [f(beta)]
        
        for i in range(nb_iterations):
            ####### Complete this part ######## or die ####################
            beta = beta - learning_rate * grad(beta)
            ###############################################################
            param_trace.append(beta)
            loss_trace.append(f(beta))
            
        return np.array(param_trace), np.array(loss_trace)
        
gd = GradientDescent()

Affichons un premier résultat d'optimisation :

In [ ]:
p, l = gd.optimize()

In [ ]:
plot_loss_contour(f, param_trace=p, three_dim=False, 
                  title='2D heatmap of the function we want to optimize')

plot_loss_contour(f, param_trace=np.concatenate([p, l.reshape((l.shape[0], 1))], axis=1), 
                  three_dim=True,  title='3D heatmap of the function we want to optimize')

L'affichage suivant interactif va nous permettre de tester les différents paramètres de notre optimiseur.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

output = widgets.Output()

@output.capture()
def interactive_gradient_descent(x, y, learning_rate, iterations):
    clear_output()
    param_trace , loss_trace = gd.optimize(nb_iterations=iterations,
                                           learning_rate=learning_rate, 
                                           beta=np.array([x, y]))
    plot_loss_contour(f, param_trace, figsize=(14.0, 6.0))
    
widgets.interact(interactive_gradient_descent,
                 learning_rate=widgets.FloatSlider(value=1e-5, min=1e-5, max=0.05, step=0.0001, 
                                                   continuous_update=False, readout_format='.5f'),
                 x=widgets.FloatSlider(value=0, min=-4, max=4, step=0.1, continuous_update=False),
                 y=widgets.FloatSlider(value=0, min=-4, max=4, step=0.1, continuous_update=False),
                 iterations=widgets.IntSlider(value=10, min=10, max=500, step=1, continuous_update=False)
)
display(output)

## II. La descente de gradient à pas optimal

Dans le scénario précédent, nous avons du fixer un pas d'optimisation $\eta$ arbitraire. Ce dernier doit être suffisament petit pour garantir que l'algorithme converge et suffisamment grand pour que l'optimisation se fasse. Il est possible de définir une notion de pas d'optimisation optimal. Cependant, celle-ci est souvent intractable en pratique (trouver le pas est plus couteux que l'optimisation initiale). Dans certains cas, nous pouvons néanmoins le déterminer. C'est ce que nous allons faire ici. Considérons maintenant la fonction suivante :

In [ ]:
A = np.array([[1, 0], [0, 2]])
b = np.array([[2], [1]])

def f(x):
    if type(x) is np.ndarray:
        x = x.tolist()
    return (np.dot(np.dot(A, x).T, x)*0.5+np.dot(b.T, x))[0]

In [ ]:
plot_loss_contour(f, three_dim=False, starting=[0, 2], 
                  title='2D heatmap of the function we want to optimize')
plot_loss_contour(f, three_dim=True, starting=[4, 4], 
                  title='3D heatmap of the function we want to optimize')

L'algorithme de descente de gradient nous permet d'avancer dans la bonne direction. Cependant, le choix du pas $\eta$ peut nous sembler insuffisant. 

Soit $\boldsymbol{\nu}=[x, y]^T$, la direction d'optimisation $\boldsymbol{d}^{(t)}=-\nabla f(\boldsymbol{\nu}^{(t)})$ et la fonction $\gamma:t\mapsto f(\boldsymbol{\nu}^{(t)}+t\boldsymbol{d}^{(t)})$. La valeur de $t$ qui minimise la fonction $\gamma$ est un pas optimal pour une minimisation dans la direction du gradient. Sans contrainte particulière sur la fonction $f$, $\gamma$ pourrait admettre un certain nombre de points critiques de natures et de valeurs différentes.

Les points critiques sont les points d'annulation de la dérivée : $\{t\in\mathbb{R}:\ \gamma^\prime(t)=0\}$. On obtient assez facilement la dérivée de la manière suivante :

$$\gamma^\prime(t)=\frac{\partial f}{\partial x}(\boldsymbol{\nu}^{(t)}+t\boldsymbol{d}^{(t)})\frac{\partial f}{\partial x}(\boldsymbol{\nu}^{(t)})+\frac{\partial f}{\partial y}(\boldsymbol{\nu}^{(t)}+t\boldsymbol{d}^{(t)})\frac{\partial f}{\partial y}(\boldsymbol{\nu}^{(t)})$$

On remarque que résoudre cette équation est rapidement problématique et nécessite l'utilisation d'un autre algorithme de descente de gradient. En réalité, il y a grossièrement deux possibilités :
1. On peut trouver une valeur $t$ analytiquement et c'est le choix qu'on doit faire,
2. Il n'est pas possible de calculer $t$ et on doit le calculer numériquement. Cependant, si on doit le calculer numériquement, alors, il devient nécessaire de calculer le gradient à chaque étape, et dans ce cas, pourquoi ne pas juste se déplacer dans l'espace des paramètres avec notre vecteur $\boldsymbol{[x, y]}$ ce qui nous donnerait une meilleure direction dans l'espace des paramètres...


Il se trouve que la fonction définie ci-dessus est : $f(\boldsymbol{x})=\frac{1}{2}\langle Ax, x\rangle+\langle b, x\rangle$ où :


$$A=\begin{bmatrix} 1& 0\\ 0& 2\end{bmatrix}$$

est symétrique définie positive et 


$$b=\begin{bmatrix}2\\1\end{bmatrix}$$

Notons 

$$\begin{aligned}
f(\boldsymbol{x}+t\boldsymbol{d})&=\frac{1}{2}\langle A(x+t\boldsymbol{d}), x+t\boldsymbol{d}\rangle+\langle b, x+t\boldsymbol{d}\rangle\\
&=\frac{1}{2}(\langle A\boldsymbol{x},\boldsymbol{x}\rangle+t\langle A\boldsymbol{d},\boldsymbol{x}\rangle+t\langle A\boldsymbol{x},\boldsymbol{d}\rangle+t^2\langle A\boldsymbol{d},\boldsymbol{d}\rangle)+\langle \boldsymbol{b},\boldsymbol{x}\rangle+t\langle \boldsymbol{b},\boldsymbol{d}\rangle\\
&=f(\boldsymbol{x})+\frac{1}{2} t^2\langle A\boldsymbol{d},\boldsymbol{d}\rangle+t\langle A\boldsymbol{x}+\boldsymbol{b},\boldsymbol{d}\rangle
\end{aligned}$$

Notons de plus que $\partial f(\boldsymbol{x})/\partial \boldsymbol{x}=A\boldsymbol{x}+\boldsymbol{b}=-\boldsymbol{d}$. Nous avons donc :

\begin{equation}
f(\boldsymbol{x}+t\boldsymbol{d})=f(\boldsymbol{x})+\frac{1}{2} t^2\langle A\boldsymbol{d},\boldsymbol{d}\rangle-t\langle \boldsymbol{d},\boldsymbol{d}\rangle=f(\boldsymbol{x})+\frac{1}{2} t^2\langle A\boldsymbol{d},\boldsymbol{d}\rangle-\left\lVert\boldsymbol{d}\right\lVert^2 t
\end{equation}

La direction $\boldsymbol{d}=-\nabla f(\boldsymbol{x})$ est celle qui indique la plus forte pente. La variable $t$ recherchée indique la taille du pas que l'on souhaite faire. Pour cela, nous devons chercher les points critiques de la fonction $\gamma(t)=f(\boldsymbol{x}+t\boldsymbol{d})$ qui sont donnés en recherchant les points d'annulation de la dérivée. De plus, $A$ (la Hessienne) étant définie positive, nous savons que ces points critiques seront des minimums. Nous avons donc :

$$\frac{\partial \gamma}{\partial t}(t)=t\langle A\boldsymbol{d}, \boldsymbol{d}\rangle - \left\lVert\boldsymbol{d}\right\lVert^2=0$$

Et le point critique est donné par :

$$t=\frac{\left\lVert\boldsymbol{d}\right\lVert^2}{\langle A\boldsymbol{d}, \boldsymbol{d}\rangle}$$

---

<span style="color:blue">**Exercice :**</span> **Donnez le code permettant de calculer le gradient (i.e. la direction d'optimisation).**



---

In [ ]:
def grad(x):
    ####### Complete this part ######## or die ####################
    return np.dot(A, x)+b
    ###############################################################

---

<span style="color:blue">**Exercice :**</span> **Donnez le code permettant de calculer une itération de l'algorithme de descente de gradient avec un pas optimal.**



---

In [ ]:
class OptimalStepGradientDescent(object):
    def optimize(self, learning_rate=0.1, nb_iterations=15, beta=None):
        if beta is None:
            beta = np.random.uniform(-2, 2, size=(2, 1))
        else:
            beta = beta.reshape((2, 1))
            
        beta2 = beta.copy()

        param_trace = [beta]
        param_trace2 = [beta2]
        loss_trace = [f(beta)]
        loss_trace2 = [f(beta2)]
        it = 0
        stop = False
        
        for i in range(nb_iterations):
            d = -grad(beta)
            if d[0, 0] == d[1, 0] == 0.:
                stop = True
            else:
                ####### Complete this part ######## or die ####################
                t = np.linalg.norm(d)**2/np.dot(np.dot(A, d).T, d)
                beta = beta + t[0, 0] * d
                ###############################################################
                
                param_trace.append(beta)
                loss_trace.append(f(beta))
            
            # cette partie du code permet de calculer le gradient classique
            # afin que nous puissions le comparer avec la descente de gradient
            # à pas optimal.
            beta2 = beta2 - learning_rate * grad(beta2)
            param_trace2.append(beta2)
            loss_trace2.append(f(beta2))
            it += 1
        return (np.array(param_trace), np.array(loss_trace), 
                np.array(param_trace2), np.array(loss_trace2))
        
gd = OptimalStepGradientDescent()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

output = widgets.Output()

@output.capture()
def interactive_gradient_descent(learning_rate, x, y, iterations):
    clear_output()
    p1, l1, p2, l2 = gd.optimize(learning_rate=learning_rate, nb_iterations=iterations,
                                           beta=np.array([x, y]))
    plot_loss_contour(f, (p1, p2), figsize=(14.0, 6.0))
    
widgets.interact(interactive_gradient_descent,
                 learning_rate=widgets.FloatSlider(value=1e-5, min=1e-5, max=1., step=0.0001, 
                                                   continuous_update=False, readout_format='.5f'),
                 x=widgets.FloatSlider(value=0, min=-4, max=4, step=0.1, continuous_update=False),
                 y=widgets.FloatSlider(value=0, min=-4, max=4, step=0.1, continuous_update=False),
                 iterations=widgets.IntSlider(value=1, min=1, max=20, step=1, continuous_update=False)
)
display(output)

## III. La méthode de Newton

### Introduction
L'élément de base de la méthode de Newton est le développement limité d'une fonction $f$ en $x_0$. Soit $f\in C^n(\mathbb{R})$ une fonction $n$ fois dérivable de dérivée $n^{eme}$ continue de $\mathbb{R}$ dans $\mathbb{R}$, son développement limité à l'ordre $n$ est donné par la formule suivante :

$$f(x)=\sum_{i=1}^n \frac{f^{(n)}(x_0)}{n!}(x-x_0)^n+o\big((x-x_0)^n\big)$$

Ainsi, la tangente à notre fonction au point $x_0$, ou approximation linéaire de notre fonction en $x_0$, est donnée par :

$$f(x)\approx f(x_0)+f^\prime(x_0)(x-x_0)$$

et l'approximation à l'ordre $2$ par :

$$f(x)\approx f(x_0)+f^\prime(x_0)(x-x_0)+\frac{f^{\prime\prime}(x_0)}{2}(x-x_0)^2$$


Ces idées se généralisent à des fonctions $f:\mathbb{R}^n\mapsto\mathbb{R}$ :

$$f(x)\approx f(x_0)+\langle \nabla f(x_0), x-x_0\rangle + \frac{1}{2} (x-x_0)^TH_f(x-x_0)$$


### La méthode
Soit $f:\mathbb{R}\mapsto\mathbb{R}$ une fonction deux fois dérivables de dérivée seconde continue, l'objectif de la méthode de Newton est de minimiser $f$ :

$$\min_{x\in\mathbb{R}}f(x)$$

Supposons de plus $f$ strictement convexe. Cette minimisation est séquentielle est produit une suite d'itérés $\{x_0, x_1, ..., x_k\}$ où $x_0$ est notre point de départ. chaque itéré se rapproche un peu plus du minimiseur recherche $x^\star$.

À chaque itérée, l'approximation à l'ordre $2$ de $f$ est elle-même une fonction (strictement) convexe et coercive. Elle admet donc un minimum que l'on peut trouver en annulant la dérivée :

$$f(x_k+t)\approx f(x_k)+f^\prime(x_k)t+\frac{f^{\prime\prime}(x_k)}{2}t^2$$

On a donc :

$$\frac{d}{dt}(f(x_k)+f^\prime(x_k)t+\frac{f^{\prime\prime}(x_k)}{2}t^2)=f^\prime(x_k)+f^{\prime\prime}(x_k)t=0\Leftrightarrow t=-\frac{f^\prime(x_k)}{f^{\prime\prime}(x_k)}$$

Ce qui nous permet de fixer l'itéré suivant : $x_{k+1}=x_k-f^\prime(x_k)/f^{\prime\prime}(x_k)$.

Cette méthode se généralise bien sûr à des fonctions à plusieurs variables comme nous allons le voir lors d'une autre section.

Considérons la fonction suivante :

In [ ]:
def f(x):
    return np.power(x-0.85, 2) + 12+np.power(x, 4)+np.exp(x)

In [ ]:
x_start = -2

In [ ]:
def plot_curve(x_start, optimization_steps=None):
    x = np.linspace(-3, 3, 301)
    plt.figure(figsize=(12.0, 8.0))
    plt.plot(x, f(x), label='Function $f$')
    plt.scatter([x_start], f(x_start), label='Optimization starting point')
    if optimization_steps is not None:
        plt.scatter(optimization_steps[:, 0], optimization_steps[:, 1], 
                    color='blue', label='Optimization steps')
    plt.title('Notre fonction $f$')
    plt.legend()
    plt.show()
plot_curve(x_start)

---

<span style="color:blue">**Exercice :**</span> **Donnez le code permettant de calculer une itération de la méthode d'optimisation de Newton. Jouez ensuite avec l'affichage interactif.**



---

In [ ]:
def f_prime(x):
    return 2*x-1.7+4*x**3+np.exp(x)

def f_prime_prime(x):
    return 2+12*x**2+np.exp(x)

class NewtonMethod(object):
    def optimize(self, x_start, nb_iterations=15):
        
        

        optimization_steps = []
        x_t = x_start
        
        for i in range(nb_iterations):
            ####### Complete this part ######## or die ####################
            x_t = x_t - f_prime(x_t)/f_prime_prime(x_t)
            ###############################################################
            
            optimization_steps.append([x_t, f(x_t)])
            
        return np.array(optimization_steps)
        
newton = NewtonMethod()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

output = widgets.Output()

@output.capture()
def interactive_gradient_descent(x, iterations):
    clear_output()
    optimization_steps = newton.optimize(x, nb_iterations=iterations)
    plot_curve(x, optimization_steps)
    
widgets.interact(interactive_gradient_descent,
                 x=widgets.FloatSlider(value=-3, min=-4, max=4, step=0.1, continuous_update=False),
                 iterations=widgets.IntSlider(value=1, min=1, max=10, step=1, continuous_update=False)
)
display(output)

## IV. La descente de coordonnées

La descente de coordonnées ou *coordinate descent* consiste à optimiser une fonction multivariée variable par variable. Soit $f:\mathbb{R}^d\mapsto\mathbb{R}$ et le problème d'optimisation suivant :

$$x^\star=\text{argmin}_{x\in\mathbb{R}^d}f(x).$$

Contrairement à la descente de gradient classique, ici, lors d'une étape d'optimisation, une unique variable est mise à jour à la fois : 

$$x^{(t)}_i=\text{argmin}_{x\in\mathbb{R}}f(x^{(t)}_1,\ldots, x_{i-1}^{(t)}, x, x_{i+1}^{(t-1)}, \ldots, x_d^{(t-1)}).$$

Considérons la fonction à deux variables suivantes : $f(x, y)=5x^2-6xy+5y^2$.

In [ ]:
def f(x):
    return 5*np.power(x[0], 2)-6*x[0]*x[1]+5*np.power(x[1], 2)

In [ ]:
plot_loss_contour(f, three_dim=False, starting=[0, 2], 
                  title='2D heatmap of the function we want to optimize')

Considérons $g_y(x)=f(x, y)$ comme une fonction de $x$ uniquement (i.e. $y$ est fixé) et dérivons :

$$g_y^\prime(x)=10x-6y.$$

Ainsi, l'itéré suivant pour la variable $x$ s'obtient de la manière suivante : $x^{(t)}=x^{(t-1)}-\eta g_y^\prime(x^{(t-1)})$. Le même raisonement s'étend de manière totalement symétrique pour obtenir l'itéré de la variable $y$ (remarquez que $f(x, y)= f(y, x)$).

---

<span style="color:blue">**Exercice :**</span> **Donnez le code permettant de calculer une itération de la méthode *coordinate descent*.**



---

In [ ]:
class CoordinateDescent(object):
    def optimize(self, x_start, nb_iterations=15, learning_rate=0.01):
        optimization_steps = [np.copy(x_start)]
        x_t = x_start
        
        for i in range(nb_iterations):
            ####### Complete this part ######## or die ####################
            x_t[0]=x_t[0]-learning_rate*(10*x_t[0]-6*x_t[1])
            ###############################################################

            optimization_steps.append(np.copy(x_t))
            
            ####### Complete this part ######## or die ####################
            x_t[1]=x_t[1]-learning_rate*(10*x_t[1]-6*x_t[0])
            ###############################################################
            
            optimization_steps.append(np.copy(x_t))

            
        return np.array(optimization_steps)
        
coordinate = CoordinateDescent()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

output = widgets.Output()

@output.capture()
def interactive_gradient_descent(learning_rate, x, y, iterations):
    clear_output()
    param_trace = coordinate.optimize(x_start=np.array([x, y]),
                                      nb_iterations=iterations,
                                      learning_rate=learning_rate)
    plot_loss_contour(f, param_trace, figsize=(14.0, 6.0))
    
widgets.interact(interactive_gradient_descent,
                 learning_rate=widgets.FloatSlider(value=1e-5, min=1e-5, max=0.1, step=0.0001, 
                                                   continuous_update=False, readout_format='.5f'),
                 x=widgets.FloatSlider(value=-2, min=-4, max=4, step=0.1, continuous_update=False),
                 y=widgets.FloatSlider(value=-2, min=-4, max=4, step=0.1, continuous_update=False),
                 iterations=widgets.IntSlider(value=1, min=1, max=20, step=1, continuous_update=False)
)
display(output)